# Viewing results in Napari

Code uses outputs created by the spot_detection script to quickly load a particular image with masks and spots into napari in order to visually evaluate the restults.

Run this on a local machine rather than the server as getting X11 forwarding to work on the server is a bit tricky. 

Pathnames from the fileList.csv df are automatically converted from /mnt/external.data/ to /Volumes/external.data/

If you are working on windows, then mount izbkingston in WSL under /mnt/izbkingston/ so it can be converted to /mnt/external.data/.

the pathnames stored in the file_list.csv of flie_list_wormMasks.csv are always in server format.

In [11]:
import napari
from skimage.measure import regionprops_table, regionprops
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import gc
from bioio import BioImage
import bioio_nd2
import bioio_tifffile
import tqdm
from bioio.writers import OmeTiffWriter
from magicgui import magicgui
from napari.settings import get_settings


settings = get_settings()
settings.application.ipy_interactive = False


anisotropy = (3,1,1) # Relative scale of (Z,X,Y) axes

nucChannel = 0 # red emerin rings
spotChannel = 0 # green spots

# in lucien's original scripts:
# channel 0 is green spots
# channel 1 is red emerin

path_type="mac" # could be "server", "mac" or "wsl"


In [ ]:
def correct_path(path):
    if path_type == "server":
        return path
    if path_type == "mac":
        newpath = path.replace('/mnt/external.data/','/Volumes/external.data/')
        return newpath
    if path_type == "wsl":
        newpath = path.replace('/mnt/external.data/','/mnt/izbkingston/')
        return newpath
    
def correct_save_path(df, col_names=['raw_filepath', 'denoised_filepath']):
    if path_type == "server":
        return df
    if path_type == "mac":
        for col in col_names:
            df[col] = df[col].str.replace('/Volumes/external.data/','/mnt/external.data/')
        return df
    if path_type == "wsl":
        for col in col_names:
            df[col] = df[col].str.replace('/mnt/izbkingston/','/mnt/external.data/')
        return df

def correct_loaded_path(df, col_names=['raw_filepath', 'denoised_filepath']):
    if path_type == "server":
        return df
    if path_type == "mac":
        for col in col_names:
            df[col] = df[col].str.replace('/mnt/external.data/','/Volumes/external.data/')
        return df
    if path_type == "wsl":
        for col in col_names:
            df[col] = df[col].str.replace('/mnt/external.data/','/mnt/izbkingston/')
        return df


#output_path = '/Volumes/external.data/MeisterLab/jsemple/lhinder/segmentation_Dario/DPY27/'
output_path_base = correct_path('/mnt/external.data/MeisterLab/jsemple/lhinder/segementation_Kalyan')

raw_input_path = correct_path('/mnt/external.data/MeisterLab/Kalyan/TF_strains/wPM1353_bet1_mSG/2025_04_03_wPM1353/2025_04_03_wPM1353_HS')
denoised_input_path = os.path.join(raw_input_path,'n2v_denoise/denoised')

if not os.path.exists(output_path_base):
    os.makedirs(output_path_base, exist_ok=True)

raw_file_name_pattern = "/*.nd2"
denoised_file_name_pattern = "/*_n2v.tif"
raw_filepaths = sorted(glob.glob(raw_input_path + raw_file_name_pattern,recursive=True))
raw_filepaths = [filepath for filepath in raw_filepaths if '_bad.nd2' not in filepath]

print(f"Found {len(raw_filepaths)} *.nd2 files.")

def kalyan_metadata(raw_input_path, raw_filepaths, output_path_base, denoised_input_path):
    # extract identifying directories from raw_input_path
    metadata = os.path.normpath(raw_input_path).split(os.sep)[-3:]
    protein = '-'.join(metadata[0].split('_')[1:])
    strain = metadata[0].split('_')[0]
    date = '-'.join(metadata[1].split('_')[:3])

    output_path = os.path.join(output_path_base, '_'.join([date,protein,strain]))
    if not os.path.exists(output_path):
        os.makedirs(output_path, exist_ok=True)

    df = pd.DataFrame()
    df['filename'] = [os.path.basename(filepath)[:-4] for filepath in raw_filepaths]
    tmpdate = [os.path.normpath(filepath).split(os.sep)[-2] for filepath in raw_filepaths]
    df['date'] = date
    df['protein'] = protein
    df['strain'] = strain
    df['treatment'] = [filename.split('_')[1] for filename in df['filename']]
    df['worm_id'] = [filename.split('_')[2] for filename in df['filename']]
    df['id'] = df['protein'] + '_' + df['date'] + '_' + df['filename'] 
    df['raw_filepath'] = raw_filepaths
    df['denoised_filepath'] = [os.path.join(denoised_input_path,filename+'_n2v.tif') for filename in df['filename']]
    if (path_type=="server"):
        df.to_csv(os.path.join(output_path,'fileList.csv'),index=False)
    return(df, output_path)

df, output_path = kalyan_metadata(raw_input_path, raw_filepaths, output_path_base, denoised_input_path)
output_path  


Found 51 *.nd2 files.


'/Volumes/external.data/MeisterLab/jsemple/lhinder/segementation_Kalyan/2025-04-03_bet1-mSG_wPM1353'

In [30]:
df.head()

,filename,date,protein,strain,treatment,worm_id,id,raw_filepath,denoised_filepath
0,wPM1353_HS_001,2025-04-03,bet1-mSG,wPM1353,HS,001,bet1-mSG_2025-04-03_wPM1353_HS_001,/Volumes/external.data/MeisterLab/Kalyan/TF_st...,/Volumes/external.data/MeisterLab/Kalyan/TF_st...
1,wPM1353_HS_002,2025-04-03,bet1-mSG,wPM1353,HS,002,bet1-mSG_2025-04-03_wPM1353_HS_002,/Volumes/external.data/MeisterLab/Kalyan/TF_st...,/Volumes/external.data/MeisterLab/Kalyan/TF_st...
2,wPM1353_HS_003,2025-04-03,bet1-mSG,wPM1353,HS,003,bet1-mSG_2025-04-03_wPM1353_HS_003,/Volumes/external.data/MeisterLab/Kalyan/TF_st...,/Volumes/external.data/MeisterLab/Kalyan/TF_st...
3,wPM1353_HS_004,2025-04-03,bet1-mSG,wPM1353,HS,004,bet1-mSG_2025-04-03_wPM1353_HS_004,/Volumes/external.data/MeisterLab/Kalyan/TF_st...,/Volumes/external.data/MeisterLab/Kalyan/TF_st...
4,wPM1353_HS_005,2025-04-03,bet1-mSG,wPM1353,HS,005,bet1-mSG_2025-04-03_wPM1353_HS_005,/Volumes/external.data/MeisterLab/Kalyan/TF_st...,/Volumes/external.data/MeisterLab/Kalyan/TF_st...


In [ ]:
def select_worm_region(df, index, output_path, spotChannel=0):
    # Load the data
    img = BioImage(df.raw_filepath.iloc[index], reader=bioio_nd2.Reader)
    img = img.get_image_data("ZYX", T=0, C=spotChannel)
    img = np.max(img[:, :, :], axis=0)

    if os.path.exists(os.path.join(output_path,'worm_masks'))==False:
        os.makedirs(os.path.join(output_path,'worm_masks'), exist_ok=True)
    
    if os.path.exists(os.path.join(output_path,'qc'))==False:
        os.makedirs(os.path.join(output_path,'qc'), exist_ok=True)

    if 'worm_region' not in df.columns:
        df['worm_region'] = ""

    # Create the Napari viewer
    viewer = napari.Viewer()

    # Add the image and shapes layers
    layer_c0 = viewer.add_image(img, blending='additive', colormap='cyan', opacity=0.85,
                            contrast_limits=[100, 250])
    layer_c1 = viewer.add_shapes(opacity=0.5, name='Worm', edge_color='white', face_color='transparent',
                             edge_width=5)

    # Dropdown widget for category selection
    @magicgui(
        auto_call=True,
        category={"choices": ["head", "tail", "tailnuc", "body", "embryo"]},  # Dropdown menu
    )
    def dropdown_widget(category: str = "head"):
        return category

    # Add a text prompt to the viewer
    viewer.text_overlay.text = "Draw worm polygon outline in 'worm' layer then press 'm' to save the mask."
    viewer.text_overlay.visible = True

    # Add the dropdown widget to the viewer
    viewer.window.add_dock_widget(dropdown_widget, area="right")

    # Key binding to save the mask
    @viewer.bind_key('m')  # Press 'm' to generate the mask
    def generate_mask(viewer):
        # Convert shapes to a binary mask
        binary_mask = layer_c1.to_masks(img.shape)
        mask_path = os.path.join(output_path, 'worm_masks', df.id.iloc[index] + '.tif')
        OmeTiffWriter.save(binary_mask.astype(np.uint8), mask_path)
        df.at[index, 'worm_region'] = dropdown_widget.category.value
        viewer.text_overlay.text = 'index'+str(index)+' '+df.at[index, 'worm_region']
        viewer.text_overlay.font_size = 40
        viewer.text_overlay.visible = True
        viewer.export_figure(os.path.join(output_path, 'qc', 'wormMask_'+df.id.iloc[index] + '.png'))
        viewer.close()

    # Run Napari
    napari.run()

    return df

 

## Manually create masks for worm region
Run this code to scroll through the rows of the df dataframe, and open a max projection of the raw image in napari. To create mask:

1) Select from dropdown menu on the top right what worm region you are highlighting (head, tail, tailnuc, body, embryo)

2) Use polygon selection tool among the options on the left menu to draw an outline around the region of interest. It does not have to be tight around the worm, just avoid any other fluorescent regions you are not interestd in. If you make a mistake you can delete and redraw.

3) When you are satisfied with your selection, press 'm' and that will save a binary mask for that region, save the region type you selected and close the napari viewer.

The code saves the df dataframe into fileList_wormMasks.csv after every image, so if your run get interrupted you can just change the range of indeces you are using.

In [33]:
if os.path.exists(os.path.join(output_path,'fileList_wormMasks.csv'))==True:
    df = pd.read_csv(os.path.join(output_path,'fileList_wormMasks.csv'))
    df = correct_loaded_path(df,col_names=['raw_filepath', 'denoised_filepath'])

indeces=range(0, len(df))

for index in tqdm.tqdm(indeces):
    df = select_worm_region(df, index, output_path, spotChannel=spotChannel)
    df_tmp = df.copy()
    df_tmp = correct_save_path(df_tmp,col_names=['raw_filepath', 'denoised_filepath'])
    df_tmp.to_csv(os.path.join(output_path,'fileList_wormMasks.csv'),index=False)




  0%|          | 0/51 [00:00<?, ?it/s]/Users/semple/micromamba/envs/lhcellpose/lib/python3.10/site-packages/dask/tokenize.py:242: UserWarning: ND2File file not closed before garbage collection. Please use `with ND2File(...):` context or call `.close()`.
  mod.loads(out, buffers=buffers)
  2%|▏         | 1/51 [00:57<47:35, 57.11s/it]/Users/semple/micromamba/envs/lhcellpose/lib/python3.10/site-packages/dask/tokenize.py:242: UserWarning: ND2File file not closed before garbage collection. Please use `with ND2File(...):` context or call `.close()`.
  mod.loads(out, buffers=buffers)
  4%|▍         | 2/51 [02:34<1:05:59, 80.81s/it]/Users/semple/micromamba/envs/lhcellpose/lib/python3.10/site-packages/dask/tokenize.py:242: UserWarning: ND2File file not closed before garbage collection. Please use `with ND2File(...):` context or call `.close()`.
  mod.loads(out, buffers=buffers)
  6%|▌         | 3/51 [03:54<1:04:13, 80.29s/it]/Users/semple/micromamba/envs/lhcellpose/lib/python3.10/site-packages/